Source: [link](https://multithreaded.stitchfix.com/blog/2016/04/19/agent-based-modeling/)

>  The concept of agent based modeling maps very well to object-oriented programming: types of agents are classes, individual agents are objects which can have states (object variables) and response methods (object methods).

In [3]:
# agent definitions
class PingPongAgent():
    def __init__(self, sound):
        self.sound = sound

    def respond(self,msg):
        if msg == 'hit_it':
            return self.sound
        return None


# model definition
class PingPongModel():
    def __init__(self):
        self.current_turn = 0
        self.agents = []
        for agent_sound in ['ping','pong']:
            self.agents.append(PingPongAgent(agent_sound))

    def simulate_timestep(self):
        output = self.agents[self.current_turn].respond('hit_it')
        self.current_turn += 1
        if self.current_turn >= len(self.agents):
            self.current_turn = 0
        return output

In [6]:
model1 = PingPongModel()

for i in range(5):
    print(model1.simulate_timestep())

ping
pong
ping
pong
ping


## Multiprocessing 
> This is an essential step in moving toward distributed computing for models with hundreds, thousands or even millions of agents.

In [7]:
import multiprocessing
import random
random.seed(0)

class PingPongAgent(multiprocessing.Process):
    def __init__(self, msg_pipe, sound):
        super(PingPongAgent, self).__init__()
        self.msg_pipe = msg_pipe
        self.sound = sound

    def run(self):
        running = True
        while running:
            msg = self.msg_pipe.recv()
            if msg == 'stop':
                running = False
            if msg == 'hit_it':
                self.msg_pipe.send(self.sound)


# model definition
class PingPongModel():
    def __init__(self):
        self.current_turn = 0
        self.agents = []
        self.msg_pipes_to_agents = []
        for agent_sound in ['ping','pong','punt','pass','play']:
            parent_conn, child_conn = multiprocessing.Pipe()
            self.msg_pipes_to_agents.append(parent_conn)
            p = PingPongAgent(child_conn, agent_sound)
            self.agents.append(p)
            p.start()

    def simulate_timestep(self):
        this_turn = random.sample(range(len(self.agents)), 
                                  int(random.uniform(0,len(self.agents))))
        for i in this_turn:
            self.msg_pipes_to_agents[i].send('hit_it')
        output = []
        for i in this_turn:
            output.append( self.msg_pipes_to_agents[i].recv() )
        return output

    def terminate_simulation(self):
        for i,a in enumerate(self.agents):
            self.msg_pipes_to_agents[i].send('stop')
            self.msg_pipes_to_agents[i].close()
            a.join()


In [8]:
model1 = PingPongModel()

for i in range(5):
    print(model1.simulate_timestep())

['pass', 'ping', 'pong', 'punt']
['punt', 'pass']
['pong']
['punt', 'pong']
['play', 'punt', 'pass']


Source: [link](http://ysar.net/python/threading-vs-multiprocessing.html)

In [10]:
from threading import Thread
from random import randint
from time import sleep

def basvuru(basvuru_id):
    sleep(randint(0,60))
    if randint(0,1):
        print("%i başvurusu kabul edildi." % basvuru_id)
    else:
        print("%i başvurusu reddedildi." % basvuru_id)


for i in range(5):
    Thread(target=basvuru, args=(i,)).start()

4 başvurusu kabul edildi.
3 başvurusu kabul edildi.
1 başvurusu reddedildi.
0 başvurusu kabul edildi.
2 başvurusu kabul edildi.
0 başvurusu kabul edildi.
1 başvurusu reddedildi.
2 başvurusu kabul edildi.
3 başvurusu kabul edildi.
4 başvurusu kabul edildi.


In [11]:
print('hell-o')

hell-o


In [14]:
from multiprocessing import Pool
from math import sqrt, floor


def asal_mi(sayi):
    "sayı asal mı?"
    if sayi < 2:
        raise ValueError("Sayi ikiden buyuk olmalidir.")

    if sayi == 2:
        return True

    if sayi % 2 == 0:
        return False

    bolunecekler = range(3, int(sqrt(sayi)),2)

    for b in bolunecekler:
        if sayi % b == 0:
            return False

    return True


def sayilar_asal_mi(sayilar):
    "Bir liste içindeki asalları döndür"
    return [x for x in sayilar if asal_mi(x)]


if __name__ == "__main__": # Bunu yapmanız şart, şimdilik niye diye sormayın.
    p = Pool(processes=4) # 4 işlemden oluşan işlemci havuzu oluştur

    # İşleri işlemler arasında dağıt
    sonuclar = p.map(sayilar_asal_mi, [ range(500000, 625000),
                                   range(625000, 750000),
                                   range(750000, 875000),
                                   range(875000, 1000000) ] )

    alt_toplamlar = map(sum, sonuclar) # Her listenin kendi toplamını al
    print(alt_toplamlar)
    print(sum(alt_toplamlar)) # tümünü topla

27670539145
